In [1]:
#保存浏览器驱动
from DrissionPage import ChromiumOptions
path = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
ChromiumOptions().set_browser_path(path).save()

配置已保存到文件: Z:\anaconda3\envs\Django\lib\site-packages\DrissionPage\_configs\configs.ini
以后程序可自动从文件加载配置


'Z:\\anaconda3\\envs\\Django\\lib\\site-packages\\DrissionPage\\_configs\\configs.ini'

In [2]:
##数据清洗

import pandas as pd

# 读取CSV文件
df = pd.read_csv('csv/cpu_products.csv')

# 定义要清洗的列名列表
columns_to_clean = ['京东价', '参考价']

for column in columns_to_clean:
    if column in df.columns:
        # 1. 去掉 '￥' 和 '¥' 符号
        df[column] = df[column].astype(str).str.replace('￥', '').str.replace('¥', '')

        # 2. 处理 '万' 字，并转换为数值
        # 检查是否包含 '万'，如果是，乘以 10000
        has_wan = df[column].str.contains('万', na=False)
        df.loc[has_wan, column] = (
            df.loc[has_wan, column]
            .str.replace('万', '')  # 去掉 '万'
            .astype(float) * 10000  # 转换为 float 并乘以 10000
        )

        # 3. 确保所有值转换为数值（非 '万' 的按原值处理）
        df[column] = pd.to_numeric(df[column], errors='coerce')
    else:
        print(f"警告：数据框中没有 '{column}' 列")

# 保存修改后的CSV文件
df.to_csv('clearcsv/cpu_history.csv', index=False)

In [1]:
import pandas as pd
import re

def parse_product_params(input_file, output_file):
    """
    解析包含产品参数的CSV文件，将参数列拆分为多列

    参数:
        input_file (str): 输入CSV文件路径
        output_file (str): 输出CSV文件路径
    """
    # 读取原始CSV文件
    df = pd.read_csv(input_file)

    # 确保存在'产品参数'列
    if '产品参数' not in df.columns:
        raise ValueError("CSV文件中没有找到'产品参数'列")

    # 预定义常见的参数键（可根据实际数据调整）
    common_params = [
        '适用类型', 'CPU系列', 'CPU主频', '最高睿频', '三级缓存',
        '插槽类型', '核心数量', '线程数',  '热设计功耗(TDP)'
    ]

    # 初始化所有参数列为空字符串
    for param in common_params:
        df[param] = ''

    # 解析每一行的产品参数
    for index, row in df.iterrows():
        if pd.isna(row['产品参数']):
            continue

        # 分割参数行
        param_lines = str(row['产品参数']).split('\n')

        # 解析每个参数键值对
        for line in param_lines:
            # 使用正则表达式匹配键值对
            match = re.match(r'^(.+?)：(.*)$', line.strip())
            if match:
                key = match.group(1).strip()
                value = match.group(2).strip()

                # 如果键在预定义列表中，则添加到DataFrame
                if key in common_params:
                    df.at[index, key] = value
                else:
                    # 对于未预定义的参数，动态添加新列
                    if key not in df.columns:
                        df[key] = ''
                    df.at[index, key] = value

    # 保存处理后的数据到新CSV文件
    df.to_csv(output_file, index=False, encoding='utf_8_sig')
    print(f"处理完成，结果已保存到: {output_file}")

# 使用示例
if __name__ == "__main__":
    input_csv = "csv/测试.csv"  # 替换为你的输入文件路径
    output_csv = "csv/out.csv"  # 输出文件路径

    parse_product_params(input_csv, output_csv)

处理完成，结果已保存到: csv/out.csv
